In [1]:
from movie_recommender import MovieRecommender
import timeit
import pandas as pd

In [2]:
started = timeit.default_timer()
movies_file = 'data/ml-10M100K/movies.dat'
ratings_file = 'data/ml-10M100K/ratings.dat'
movie_rec = MovieRecommender(movies_filename=movies_file, ratings_filename=ratings_file)
elapsed = timeit.default_timer() - started
print(f'Read dataframes in {elapsed}ms')

print(movie_rec.ratings_df['user_id'].nunique())

found pickle at c:\Users\john.french\movie_recs\data\pickled\movie_ratings.pkl
Read dataframes in 20.934671099996194ms
69878


In [3]:
# replace pipe character with space
movie_rec.movies_df['genres'] = movie_rec.movies_df['genres'].str.replace('|', ' ')
rating_counts = movie_rec.ratings_df.groupby('movie_id').agg('rating').count()
rating_counts.name = 'num_ratings'
rating_avg = movie_rec.ratings_df.groupby('movie_id').agg('rating').mean()
rating_avg.name = 'avg_rating'
most_rated = rating_counts.sort_values(ascending=False).head(1000)
most_rated = pd.merge(most_rated, movie_rec.movies_df, on="movie_id")
most_rated = pd.merge(rating_avg, most_rated, on="movie_id")
most_rated
most_rated.sort_values(by='num_ratings', ascending=False, inplace=True)

C:\Users\john.french\AppData\Local\Temp\ipykernel_19180\2230656454.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movie_rec.movies_df['genres'] = movie_rec.movies_df['genres'].str.replace('|', ' ')


In [4]:
ctr = 0
limit = 15
for idx, row in most_rated.head(10).iterrows():
    print(row['name'], idx)
    ctr += 1
    if ctr > limit:
        break

Pulp Fiction (1994) 296
Forrest Gump (1994) 356
Silence of the Lambs, The (1991) 593
Jurassic Park (1993) 480
Shawshank Redemption, The (1994) 318
Braveheart (1995) 110
Fugitive, The (1993) 457
Terminator 2: Judgment Day (1991) 589
Star Wars: Episode IV - A New Hope (a.k.a. Star Wars) (1977) 260
Apollo 13 (1995) 150


In [5]:
from ai_posters import test_api_route

ctr = 0
offset = 749
limit = 800
for idx, row in most_rated.iterrows():
    movie_title = row['name']
    genres = row['genres']
    movie_id = idx
    ctr += 1
    if ctr >= offset:
        test_api_route.make_poster_for_movie(movie_title, genres, str(movie_id))
    if ctr == limit:
        break

<Response [200]>
3371556394
Images saved successfully
<Response [200]>
241884115
Images saved successfully
<Response [200]>
3752983517
Images saved successfully
<Response [200]>
1325528479
Images saved successfully
<Response [200]>
2993825427
Images saved successfully
<Response [200]>
2646122744
Images saved successfully
<Response [200]>
506436730
Images saved successfully
<Response [200]>
582829377
Images saved successfully
<Response [200]>
776686833
Images saved successfully
<Response [200]>
3731331559
Images saved successfully
<Response [200]>
317969947
Images saved successfully
<Response [200]>
1949234268
Images saved successfully
<Response [200]>
1779135627
Images saved successfully
<Response [200]>
767568456
Images saved successfully
<Response [200]>
3860918895
Images saved successfully
<Response [200]>
2210697445
Images saved successfully
<Response [200]>
2258046341
Images saved successfully
<Response [200]>
382860224
Images saved successfully
<Response [200]>
2918185438
Images 

In [6]:
# clean_titles = [movie_rec.get_movie_title(m) for m in set(movie_rec.ratings_df.index.values)]
import pandas as pd

# create a DataFrame containing the movie titles and ids
movies_df = pd.DataFrame({'title': movie_rec.movies_df['name']})

# count the occurrences of each title using the value_counts() method
title_counts = movies_df['title'].value_counts()

# filter out the titles that appear only once using boolean indexing
popular_titles = title_counts[title_counts > 1].index.tolist()

# get the corresponding movie ids for the popular titles
popular_movie_ids = movies_df.index.get_level_values('movie_id')[movies_df['title'].isin(popular_titles)].unique().tolist()

print(popular_movie_ids)

[34048, 64997]


In [7]:
u = int(movie_rec.mrm_df.shape[0])
# u = 2
# print(type(u), u)

# movie_rec.new_add_movie_rating(movie_id=1219, user_id=u, rating=5) # Psycho
# How to make an american quilt
# movie_rec.new_add_movie_rating(movie_id=46, user_id=u, rating=5) 
# movie_rec.new_add_movie_rating(movie_id=16, user_id=u, rating=5) # Casino
# movie_rec.new_add_movie_rating(movie_id=2596, user_id=u, rating=5) # Rocky Horror
# movie_rec.new_add_movie_rating(movie_id=2571, user_id=u, rating=5) # The Matrix
movie_rec.new_add_movie_rating(movie_id=107, user_id=u, rating=5) # Muppet Treasure Island
movie_rec.new_add_movie_rating(movie_id=5080, user_id=u, rating=5) # Slackers
# movie_rec.new_add_movie_rating(movie_id=30, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=232, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=858, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=48, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=1213, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=858, user_id=u, rating=5)

movie_rec.refit_model()


started add_movie_rating with user of 69878
(69878, 10677)
movie_id  1      2      3      4      5      6      7      8      9      \
69878         0      0      0      0      0      0      0      0      0   

movie_id  10     ...  65006  65011  65025  65027  65037  65088  65091  65126  \
69878         0  ...      0      0      0      0      0      0      0      0   

movie_id  65130  65133  
69878         0      0  

[1 rows x 10677 columns]
(69879, 10677)
started add_movie_rating with user of 69878
(69879, 10677)
movie_id  1      2      3      4      5      6      7      8      9      \
69878         0      0      0      0      0      0      0      0      0   
69878         0      0      0      0      0      0      0      0      0   

movie_id  10     ...  65006  65011  65025  65027  65037  65088  65091  65126  \
69878         0  ...      0      0      0      0      0      0      0      0   
69878         0  ...      0      0      0      0      0      0      0      0   

movie_id  65

In [8]:
movie_rec.get_user_based_recommendation(u)

get user recommendation started for 69878
get similar users called on user  69878  with type  <class 'int'>
get_similar_users called with self.mrm_df shape of (69880, 10677)
values for 69878
knn_input with shape of (2, 10677)
get_similar_users called with knn_input shape of (2, 10677)
Top 3 users who are very much similar to the User- 69878are: 
1 . User: 30923 separated by distance of 0.6518446880886043
2 . User: 69824 separated by distance of 0.6772513878160487
3 . User: 35016 separated by distance of 0.6876524762227878
Muppet Treasure Island
Traffic
Grosse Pointe Blank
Arachnophobia
Short Cuts
James and the Giant Peach


In [9]:
# movie_rec.get_user_movies(69877)
for m in movie_rec.get_user_movies(69877):
    print(movie_rec.get_movie_title(m), movie_rec.get_user_movie_rating(69877, m))
# movie_rec.get_user_movie_rating(8546, 1)
# movie_rec.get_user_movie_rating(8546, 2)
# print(f'User {u} has seen...')
# for m in movie_rec.get_user_movies(u):
#     print(movie_rec.get_movie_title(m))

Lost World: Jurassic Park, The (Jurassic Park 2) 2
Starship Troopers 5
Schindler's List 4
Close Encounters of the Third Kind 5
Maltese Falcon, The 4
Eyes Wide Shut 5
Time Bandits 4
Starsky & Hutch 2
So I Married an Axe Murderer 4
Front, The 4
To Die For 4
Brazil 5
Lawrence of Arabia 5
Cast Away 5
Lost in Translation 5
Almost Famous 3
Eternal Sunshine of the Spotless Mind 5
Croupier 3
Hellboy 4
Erin Brockovich 1
Shrek 2
Pinocchio 4
Big Bounce, The 1
Glory 3
Rushmore 5
City Lights 5
Naked Gun 33 1/3: The Final Insult 2
Sneakers 4
Christmas Story, A 5


In [10]:
users, dists = movie_rec.get_similar_users(u)
# for rec_user in users:
#     # print(f'{rec_user}\'s movies')
#     for m in movie_rec.get_user_movies(rec_user):
#         # print(movie_rec.get_movie_title(m))

# print(movie_rec.get_movie_title(20))
# print(movie_rec.get_movie_title(25))
# print(movie_rec.get_movie_title(50))

get similar users called on user  69878  with type  <class 'int'>
get_similar_users called with self.mrm_df shape of (69880, 10677)
values for 69878
knn_input with shape of (2, 10677)
get_similar_users called with knn_input shape of (2, 10677)
Top 3 users who are very much similar to the User- 69878are: 
1 . User: 30923 separated by distance of 0.6518446880886043
2 . User: 69824 separated by distance of 0.6772513878160487
3 . User: 35016 separated by distance of 0.6876524762227878


In [11]:
user_id = movie_rec.get_next_avail_user_id()
# movie_rec.add_movie_rating(movie_id=1, user_id= user_id, rating= 5) # Toy Story
# movie_rec.add_movie_rating(movie_id=2, user_id= user_id, rating= 5) # Jumanji
# movie_rec.add_movie_rating(movie_id=48, user_id= user_id, rating= 5) # Pocahontas
# movie_rec.add_movie_rating(movie_id=20, user_id= user_id, rating= 5) # Money Train
# movie_rec.add_movie_rating(movie_id=25, user_id= user_id, rating= 5) # Leaving Las Vegas
# movie_rec.add_movie_rating(movie_id=50, user_id= user_id, rating= 5) # The Usual Suspects
movie_rec.ratings_df.head()

,user_id,rating,timestamp
movie_id,,,
122,1,5,838985046.0
185,1,5,838983525.0
231,1,5,838983392.0
292,1,5,838983421.0
316,1,5,838983392.0


In [12]:
import numpy as np


# return the difference between average ratings for two movies
def get_movie_mean_rating_error(movie1_id: int, movie2_id: int):
    movie1_ratings = np.mean(
        movie_rec.ratings_df.loc[movie_rec.ratings_df.index == movie1_id]['rating'].values)
    movie2_ratings = np.mean(
        movie_rec.ratings_df.loc[movie_rec.ratings_df.index == movie2_id]['rating'].values)
    return abs(movie1_ratings - movie2_ratings)


# get_movie_mean_rating_error(599, 700)
# movie_rec.get_movie_users(5)
print(movie_rec.get_movie_title(1))
print(movie_rec.get_movie_title(2))
print(len(movie_rec.get_movie_users(1)))
print(len(movie_rec.get_movie_users(2)))
print(len(movie_rec.get_common_users(1, 2)))

Toy Story
Jumanji
26449
12032
7394


In [13]:
def get_movie_rating_pearson_coeff(movie1_id: int, movie2_id: int)-> float:
    common_users = movie_rec.get_common_users(movie1_id=movie1_id, movie2_id=movie2_id)
    movie1_ratings = [movie_rec.get_user_movie_rating(user_id=u, movie_id=movie1_id) for u in common_users]
    movie2_ratings = [movie_rec.get_user_movie_rating(user_id=u, movie_id=movie2_id) for u in common_users]
    coeff = np.corrcoef(np.array(movie1_ratings), np.array(movie2_ratings))[0, 1]
    movie1_title = movie_rec.get_movie_title(movie1_id)
    movie2_title = movie_rec.get_movie_title(movie2_id)
    print(f'{movie1_title} and {movie2_title} have a correlation of {"{:.2f}".format(coeff)} from {len(common_users)} users who have rated both')

get_movie_rating_pearson_coeff(1, 3)

Toy Story and Grumpier Old Men have a correlation of 0.19 from 5106 users who have rated both
